In [32]:
import pandas as pd
import xmlrpc.client

In [33]:
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


# 1. Obtener las rutas de produccion
rutas_produccion = models.execute_kw(
    db, uid, password,
    'mrp.routing.workcenter', 'search_read',
    [[]],   
    {'fields': ['id', 'workcenter_id','sequence','display_name','efficiency_time','hour_nbr','routing_id','tota_nbr_minimo']}
)


df_rutas = pd.DataFrame(rutas_produccion) # productos con costo entre los manufacturados

# Separar store_id
df_rutas["workcenter"] = df_rutas["workcenter_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)
df_rutas["referencia"] = df_rutas["routing_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)


df_rutas_clean = df_rutas.drop(columns=["routing_id","workcenter_id"])

#Tarda descargando aprox 1 minuto

In [34]:
df_rutas_clean = df_rutas_clean[["id","referencia","sequence","workcenter","display_name","hour_nbr","efficiency_time"]]

In [35]:
df = df_rutas_clean.rename(columns={"referencia":"producto","display_name":"tarea","hour_nbr":"segundos_promedio"})

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6247 entries, 0 to 6246
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 6247 non-null   int64  
 1   producto           6247 non-null   object 
 2   sequence           6247 non-null   int64  
 3   workcenter         6247 non-null   object 
 4   tarea              6247 non-null   object 
 5   segundos_promedio  6247 non-null   float64
 6   efficiency_time    6247 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 341.8+ KB


In [37]:
df["tarea"].unique()
df["tarea"].nunique()

48

In [38]:
df["workcenter"].unique()

array(['CORTE', 'EMBALAJE', 'PLISADO', 'INYECCION', 'INTERNOS GUATA'],
      dtype=object)

In [39]:
df["workcenter"] = df["workcenter"].replace("INTERNOS GUATA","PLISADO")
df["tarea"] = df["tarea"].replace("EMBALAJE","EMPAQUE")

df["tarea"].unique()

array(['TAPAS', 'BOLSA', 'POBLAN', 'ENTUBADORA 3', 'CORTE PLASMA',
       'DESARROLLO Y ALTURA', 'DEFLECTOR', 'INSERTO', 'ESLAVAS',
       'ENTUBADORA 2', 'ENTUBADORA 1', 'ENTUBADORA 4', 'REFUERZO',
       'REPUJADO', 'TAPAS Y MALLA', 'XPEC', 'PLISADORA', 'ALTURA',
       'DESPUNTE Y ENROLLADO', 'FABCAR', 'SOLDADURA',
       'TROQUELADO Y DOBLADO', 'FRESADORA', 'PLISADORA 1', 'INYECCION',
       'GUATA', 'TROQUELADO', 'OMS', 'PLISADORA 3',
       'MARCADO/HOTMELT/ARANDELA/EMPAQUE', 'GRAFADORA', 'PLISADORA ',
       'HENNECKE', 'TORNILLOS', 'INYECCION MESA INTERNOS',
       'TERMOSELLADORA', 'TECNONEXT DURO', 'PLISADORA 4', 'TECNONEXT',
       'CONTROL DE CALIDAD', 'TECNONEXT NEGRO', 'EMPAQUE',
       'TECNONEXT NARANJA', 'TENCNONEXT NARANJA', 'TENONEXT',
       'TECNONEXT ', 'TECNONEXT NARANJA ', 'INYECCION NARANJA'],
      dtype=object)

In [40]:
df["Seleccion"] =   if df["workcenter"]

ALTURA
CORTE
CORTE PLASMA
DEFLECTOR
DESARROLLO Y ALTURA
DESPUNTE Y ENROLLADO
ENTUBADORA
ENTUBADORA 1
ENTUBADORA 2
ENTUBADORA 3
ENTUBADORA 4
ESLAVAS
FABCAR
FRESADORA
GRAFADORA
INSERTO
POBLAN
REFUERZO
REPUJADO
SOLDADURA
TAPAS
TAPAS Y MALLA
TORNILLOS
TROQUELADO
TROQUELADO Y DOBLADO
VENTANA

BOLSA
CONTROL DE CALIDAD
EMBALAJE
EMPAQUE
HOTMELT
MARCADO/HOTMELT/ARANDELA/EMPAQUE

GUATA
TERMOSELLADORA

HENNECKE
INYECCION
INYECCION MESA INTERNOS
INYECCION NARANJA
OMS
POLIURETANO
TECNONEXT
TECNONEXT DURO
TECNONEXT NARANJA
TECNONEXT NEGRO
TENCNONEXT NARANJA
TENONEXT

PLISADORA
PLISADORA 1
PLISADORA 3
PLISADORA 4
PLISADORS 3
TECNONEXT
XPEC

SyntaxError: invalid syntax (3508501514.py, line 1)